# 第7章：构建聊天应用
## Azure OpenAI API 快速入门

## 概述
该笔记本是从[Azure OpenAI示例存储库](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)改编而来，其中包括还访问[OpenAI](notebook-openai.ipynb)服务的笔记本。

Python OpenAI API 也适用于 Azure OpenAI，并进行了一些修改。了解更多关于这些区别的信息：[如何在 Python 中在 OpenAI 和 Azure OpenAI 之间切换端点](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

有关更多快速入门示例，请参考官方[Azure Open AI 快速入门文档](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)。

## 目录  

[概述](#overview)  
[使用 Azure OpenAI 服务入门](#getting-started-with-azure-openai-service)  
[构建你的第一个提示](#build-your-first-prompt)  

[用例](#use-cases)    
[1. 文本摘要](#summarize-text)  
[2. 文本分类](#classify-text)  
[3. 生成新产品名称](#generate-new-product-names)  
[4. 对分类器进行微调](#fine-tune-a-classifier)  
[5. 嵌入](#embeddings)

[参考](#references) 

### 开始使用Azure OpenAI服务

新客户需要[申请访问权限](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst)来使用Azure OpenAI服务。
申请通过后，客户可以登录Azure门户，创建Azure OpenAI服务资源，并通过工作室开始尝试不同的模型。

[快速入门的绝佳资源](https://techcommunity.microsoft.com/t5/educator-developer-blog/azure-openai-is-now-generally-available/ba-p/3719177?WT.mc_id=academic-105485-koreyst)

### 构建你的第一个提示

这个简短的练习将为你介绍如何向OpenAI模型提交用于简单任务“summarization（总结）”的提示。

![](images/generative-AI-models-reduced.jpg)

**步骤**：
1. 在你的Python环境中安装OpenAI库。
2. 加载标准辅助库，并设置你通常用于你所创建的OpenAI服务的安全凭据。
3. 为你的任务选择一个模型。
4. 为模型创建一个简单的提示。
5. 将你的请求提交到模型API！

To install the OpenAI package, use the following code:

```bash
pip install openai
```

After the installation is completed, you can import and use the OpenAI package in your Python code.


>[!注意] 如果在Codespaces上运行此笔记本或在Devcontainer中运行，则此步骤是不必要的。

In [ ]:
%pip install openai python-dotenv

```python
# Import the necessary libraries
import pandas as pd
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Replace 'your_apikey' and 'your_url' with your API key and URL
authenticator = IAMAuthenticator('your_apikey')
assistant = AssistantV2(
    version='2021-06-14',
    authenticator=authenticator
)
assistant.set_service_url('your_url')
```
### 2. 导入辅助库并实例化凭证

```python
# 导入必要的库
import pandas as pd
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# 用你的API密钥和URL替换'your_apikey'和'your_url'
authenticator = IAMAuthenticator('your_apikey')
assistant = AssistantV2(
    version='2021-06-14',
    authenticator=authenticator
)
assistant.set_service_url('your_url')
```


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )


### 3. 寻找合适的模型  
GPT-3.5-turbo 或 GPT-4 模型可以理解并生成自然语言。该服务提供了四种模型能力，每种能力都具有不同的强度和速度，适用于不同的任务。

[Azure OpenAI 模型](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
![](images/a-b-c-d-models-reduced.jpg)

In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. 提示设计  

大型语言模型的魔力在于通过在大量文本上进行训练以最小化预测错误，模型最终学会了对这些预测有用的概念。例如，它们学会了如下概念：

- 如何拼写
- 语法的工作原理
- 如何改写
- 如何回答问题
- 如何进行对话
- 如何用多种语言书写
- 如何编码
- 等等

#### 如何控制大型语言模型  
"对于大型语言模型的所有输入中，最具影响力的是文本提示(1)。

可以通过以下几种方式来提示大型语言模型生成输出：

- 指导：告诉模型你想要什么
- 完整：诱导模型完成你想要的开头部分
- 演示：向模型展示你想要的内容，可以是：
- 在提示中提供少量示例
- 在微调训练数据集中提供成百上千个示例


#### 创建提示的三个基本指导原则：

**展示和告诉**。清楚地表明你想要什么，可以通过指导、示例或两者的组合来完成。如果你希望模型对一个项目列表按字母顺序进行排名，或者对一个段落进行情感分类，清楚地告诉模型这就是你想要的。

**提供高质量数据**。如果你试图构建一个分类器或让模型遵循某种模式，请确保有足够的示例。务必校对你的示例——模型通常足够智能，可以纠正基本的拼写错误并给出响应，但它也可能会认为这是故意的，从而影响响应结果。

**检查你的设置。** 温度和top_p设置控制模型在生成响应时的确定性程度。如果你要求模型给出只有一个正确答案的响应，那么你需要把这些设置调低。如果你希望获取更多样化的响应，那么你可能需要将它们调高。人们在使用这些设置时最常犯的错误是认为它们可以控制“聪明”或“创造力”。

来源: https://github.com/Azure/OpenAI/blob/main/How%20to/Completions.md

![](images/prompt_design.jpg)
创建你的第一个文本提示！

### 5. 提交！

In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### 重复相同的调用，结果如何比较？

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## 文本摘要  
#### 挑战  
通过在文本段落末尾添加'tl;dr:'来总结文本。注意模型如何理解如何执行多项任务而无需额外说明。你可以尝试使用比'tl;dr'更具描述性的提示词来修改模型的行为，并定制您收到的摘要(3)。  

最近的研究表明，通过在大量文本语料库上进行预训练，然后在特定任务上进行微调，许多自然语言处理任务和基准测试取得了实质性的进展。虽然通常在体系结构上不受特定任务的影响，但这种方法仍然需要数千或数万个示例的特定任务微调数据集。相比之下，人类通常可以仅通过少量示例或简单的说明执行新的语言任务，而当前的自然语言处理系统在这方面仍然很大程度上存在困难。我们在这里展示，扩大语言模型能够极大提高通用任务的少示例性能，有时甚至能够达到先前的最先进微调方法的竞争力。  

Tl;dr:  通过扩大语言模型可大大提高通用任务的少示例性能。

# 几种用例的练习

## 1. 文本摘要
在该练习中，你将使用不同的提示词，练习提示工程来总结给定文本。

## 2. 文本分类
通过使用不同的提示来构建一个文本分类模型，以对文本进行分类。

## 3. 生成新的产品名称
尝试使用零示例、单示例或少示例的思维链来生成新的产品名称。

## 4. 嵌入
学习如何使用嵌入技术将文本转换成向量表示，以便进行后续的分析或处理。

## 5. 微调分类器
通过微调已有的分类器，来适应特定领域或任务的需求。

In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 文本分类  
#### 挑战  
在推理时将项目分类。 在下面的示例中，我们在提示(*playground_reference)中提供了类别和要分类的文本。

客户咨询：你好，我的笔记本键盘上的一个键最近坏了，我需要换一个。

分类的类别：

In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 生成新产品名称
#### 挑战
从示例词中创建产品名称。在此，我们在提示中包含了关于我们将要为其生成名称的产品的信息。我们还提供了一个类似的示例，以显示我们希望接收到的模式。我们还设置了较高的温度值，以增加随机性和更有创意的响应。

产品描述：家用奶昔制造机
种子词：快速，健康，紧凑
产品名称：家用摇摇杯，健康摇摇杯，快速奶昔，摇摇机制造器

产品描述：可以适应任何脚型的一双鞋
种子词：适应性强，合身，全尺寸适应

In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## 嵌入

这一部分将展示如何检索嵌入，并找到单词、句子和文档之间的相似性。为了运行以下笔记本，您需要部署一个使用`text-embedding-ada-002`作为基础模型的模型，并在.env文件中设置其部署名称，使用`AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT`变量。

### 模型分类法 - 选择嵌入模型

**模型分类法**：{家族} - {能力} - {输入类型} - {标识符}

{家族}     --> 文本嵌入  (嵌入家族)
{能力} --> ada             (所有其他嵌入模型将在2024年退役)
{输入类型} --> 不适用            (仅对搜索模型进行了指定)
{标识符} --> 002             (版本002)

model = 'text-embedding-ada-002'


> [! 注意] 如果在 Codespaces 上运行此笔记本或在 Devcontainer 中运行，以下步骤是不必要的。

In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

# 比较CNN每日新闻数据集中的文章
来源：https://huggingface.co/datasets/cnn_dailymail

该数据集由CNN每日新闻和《每日邮报》提供，包含了数百万篇新闻文章及其摘要。这些文章涵盖了各种各样的主题，是用于进行大规模自然语言处理任务的绝佳资源。

In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"' 
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()                      

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# 参考资料  
- [Azure 文档 - Azure Open AI 模型](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio 示例](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)

# 寻求更多帮助  
[OpenAI 商业化团队](AzureOpenAITeam@microsoft.com)  
AI 专业服务代表 [aka.ms/airangers](https://aka.ms/airangers?WT.mc_id=academic-105485-koreyst)

# 贡献者
* Louis Li